In [ ]:
# Notes:
# - use modified LSR from Dept Gen
# - ensure all cities geocoded
# This is a good use for virtual environments: make a requirements.txt, make an env, keep code safe even after upgrades

In [ ]:
%matplotlib agg

import city_counts
import datetime
import math
from matplotlib import rcParams
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
# Constants
THIS_YEAR = '2018-19'
COLOR_LIST = ['#FFFFFF', '#A4BCC4', 'red', 'green', 'blue', 'cyan', 'magenta']
rcParams['font.family'] = 'Calibri'

In [ ]:
# Functions
# Empty subplot
def clear_subplot(ax):
    ax.axis('off')

# Get picture
def get_pic(file, resize_factor=1):
    pic = Image.open(file)
    width  = math.floor(pic.size[0] / resize_factor)
    height = math.floor(pic.size[1] / resize_factor)
    pic = pic.resize((width, height), resample=Image.ANTIALIAS)
    return pic, width, height

# Add point to map
def add_point(my_map, lat, lng, color, markersize=20):
    my_map.plot(x=lng, y=lat, color=color, markersize=markersize,
                marker='o', alpha=0.7, latlon=True)

In [ ]:
def build_map(dept_code, cities):
    # Build layout, landscape legal aspect ratio
    fig = plt.figure()
    fig.set_figheight(12.75)
    fig.set_figwidth(21)
    fig.set_facecolor(COLOR_LIST[0])
    
    # ax0 and ax2 left empty to leave room for logos
    ax0 = plt.subplot2grid((18, 6), (0, 0), rowspan=2, colspan=6)
    ax1 = plt.subplot2grid((18, 6), (2, 0), rowspan=15, colspan=6)
    ax2 = plt.subplot2grid((18, 6), (17, 0), rowspan=1, colspan=6)
    clear_subplot(ax0)
    clear_subplot(ax2)
    
    # Add FIPS
    # Top left
    csps, _, height = get_pic('FIPs/csps.png', resize_factor=2.5)
    fig.figimage(csps, xo=0, yo=fig.bbox.ymax - height, origin='upper', zorder=1)
    
    # Bottom left
    my_school, _, _ = get_pic('FIPs/my_school.png', resize_factor=1.2)
    fig.figimage(my_school, xo=0, yo=0, origin='upper', zorder=1)
    
    # Bottom right
    canada, width, _ = get_pic('FIPs/canada.png', resize_factor=2.5)
    fig.figimage(canada, xo=fig.bbox.xmax - width, yo=0, origin='upper', zorder=1)
    
    # Add text
    ax2.text(x=0.5, y=0.0, horizontalalignment='center', fontsize=12, 
             s='Prepared by Curriculum Management – ' + \
               'Préparé par Gestion du Curriculum')
    ax2.text(x=0.9, y=0.5, horizontalalignment='right', fontsize=12, 
             s='See Appendix 1 for Methodology – ' + \
               'Consulter l\'Annexe 1 pour la méthodologie')
    ax2.text(x=0.9, y=0.0, horizontalalignment='right', fontsize=12, 
             s='Report generated on – ' + \
               'Rapport généré le ' + datetime.datetime.today().strftime('%d/%m/%Y'))
    ax2.text(x=0.9, y=-0.5, horizontalalignment='right', fontsize=12, 
             s='Page 3/3')
    
    # Map's axis
    ax1.set_title('Curriculum Usage Map, {} – '.format(THIS_YEAR) + \
                  'Carte de participation au cursus, {}'.format(THIS_YEAR) + '\n' + \
                  'Learners Attending Courses and Events In-Person – ' + \
                  'Apprenants assistants à des cours et évènments en personne',
                  fontsize=20, fontweight='bold')
    ax1.set_facecolor(COLOR_LIST[0])
    ax1.set_xticks([])
    
    # Map
    map1 = Basemap(projection='merc', urcrnrlat=70.0, urcrnrlon=-52.0,
                   resolution='l',    llcrnrlat=41.0, llcrnrlon=-140.0, ax=ax1)
    map1.drawmapboundary(fill_color=COLOR_LIST[1])
    map1.fillcontinents(color=COLOR_LIST[0], lake_color=COLOR_LIST[1])
    map1.drawcoastlines()
    map1.drawcountries()
    map1.drawstates()
    
    # Add legend
    tiny = mpatches.Patch(color=COLOR_LIST[2], label='1-10')
    small = mpatches.Patch(color=COLOR_LIST[3], label='11-50')
    medium = mpatches.Patch(color=COLOR_LIST[4], label='51-100')
    large = mpatches.Patch(color=COLOR_LIST[5], label='101-500')
    xlarge = mpatches.Patch(color=COLOR_LIST[6], label='501+')
    ax1.legend(handles=[tiny, small, medium, large, xlarge], loc='lower left',
               facecolor=COLOR_LIST[0], framealpha=1.0,
               title='No. Learners - No. d\'apprenants')
    
    # Add points
    for color, lat, lng in zip(cities['city_prov'], cities['Latitude'], cities['Longitude']):
        if not np.isnan(lat):
            add_point(map1, float(lat), float(lng), color)
    
    # Output to PDF; use tight layout to avoid overlap
    fig.tight_layout(pad=1.0, h_pad=1.0)
    fig.savefig('D:\\Maps\\PDFs 2\\{}.pdf'.format(dept_code))
    
    # Clear fig, ax, and map
    fig.clear()
    plt.close('all')

In [ ]:
departments = pd.read_csv('D:\\Maps\\Data\\departments.csv')
departments = departments['Department'].tolist()

In [ ]:
%%time
for department in departments:
    build_map(department, city_counts.city_counts(department))